In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene("../../scenes/accenture_5G_indoor/Acccenture_inside_building.xml")

In [ ]:
scene.get("elm__6").radio_material = "itu_glass"
scene.get("elm__7").radio_material = "itu_brick"
scene.get("elm__8").radio_material = "itu_concrete"
scene.get("elm__9").radio_material = "itu_metal"
scene.get("elm__10").radio_material = "itu_concrete"

In [ ]:
epsilon_r_complex = scene.get("elm__6").radio_material.complex_relative_permittivity

In [ ]:
print(epsilon_r_complex)
k = np.imag(np.sqrt(epsilon_r_complex))
n = np.real(np.sqrt(epsilon_r_complex))
n,k

In [ ]:
R = ((1-n)**2+k**2)/((1+n)**2+k**2)
R

In [ ]:
xy_center = [0,16]
z_centers = [5.6, 1.5]

In [ ]:
sampling_frequency = 10e6
sample_m = 62.0 / sampling_frequency

c0 = sionna.SPEED_OF_LIGHT

fc = 3.7e9
scene.frequency = fc # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [ ]:
epsilon_0 = 8.854 * 10**-12
T = 1-R
alpha = (2 * 2 * np.pi * fc * np.abs(k))/c0
print(T, alpha)
T * (1 - np.e ** -(alpha*0.03)) * T

In [ ]:
da = 0.1
M = 4
N = 4

scene._clear()

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=N,
                             num_cols=M,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="dipole",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="dipole",
                             polarization="V")
                             #polarization="cross")

In [ ]:
# transmitters = [
#     {
#         "name":"main_entrance",
#         "position":[-20.1,8.91,2.8],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
#     {
#         "name":"idk",
#         #"position":[5.5,16.5,3],
#         "position":[-1.5,12,3],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
#     {
#         "name":"glass_room",
#         "position":[-20.3,20.5,3],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
#     {
#         "name":"room_left",
#         "position":[-14.4553, 22,3],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
#     #[-17,18.53,2.8]
#     {
#         "name":"next_glass",
#         "position":[-18.11,20.76,3],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
#     {
#         "name":"room_next_to_cars",
#         "position":[-17.45,8.63,3],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
#     {
#         "name":"next_to_cars",
#         "position":[-13.5,14.5,3],
#         "orientation":[0.0, 0.0, 0.0],
#         "antenna_type":"dipole"
#     },
# ]

transmitters = [
    {
        "name":"main_entrance",
        "position":[-30,24.21,3],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"idk",
        #"position":[5.5,16.5,3],
        "position":[-1.5,22.6,3],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"glass_room",
        "position":[-21,24.8,3],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"room_left",
        "position":[-5, 25.5,3],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    #[-17,18.53,2.8]
    {
        "name":"next_glass",
        "position":[-20.04,19.31,2.8],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"room_next_to_cars",
        "position":[-16,10.5,3],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"next_to_cars",
        "position":[-5.5,14.5,3],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
]


receivers = [
    {
        "name":"rx_outdoor_1",
        "position":[43.9394, 125.376, 1.5+5.56987],
        "orientation":[0,0,0]
    },
    {
        "name":"rx_outdoor_2",
        "position":[153, 125.376, 1.5+5.56987],
        "orientation":[0,0,0]
    },
    {
        "name":"rx_indoor_1",
        "position":[94,20, 1.5],
        "orientation":[0,0,0]
    },
    {
        "name":"rx_indoor_2",
        "position":[94,20, 1.5],
        "orientation":[0,0,0]
    }
]

In [ ]:
for i in range(len(transmitters)):
    tx = Transmitter(name=transmitters[i]["name"],
                 position=transmitters[i]["position"])
    # rx = Receiver(name=receivers[i]["name"],
    #           position=receivers[i]["position"])
    scene.add(tx)
    print(transmitters[i]["name"])
    #scene.add(rx)

In [ ]:
import numpy as np
import tensorflow as tf

# Configure an SGD optimizer
optimizer = tf.keras.optimizers.legacy.RMSprop(0.1)

# Number of training steps
num_steps = 1

# Weighting factors for the loss terms
weight_sinr = 1
weight_coverage = 0
a = tf.Variable(250, dtype=tf.float32)
b = tf.Variable(250, dtype=tf.float32)
c = tf.Variable(250, dtype=tf.float32)
d = tf.Variable(250, dtype=tf.float32)

def train_step():
    """A single training step"""
    with tf.GradientTape() as tape:
        
        # Compute coverage of the target area
        target_cm = scene.coverage_map(
                                       cm_center=xy_center + [z_centers[1]],
                                       cm_orientation=[0,0,0],
                                       cm_size=[100,50],
                                       cm_cell_size=[0.1,0.1])
        
        # Convert to tf.float32 to ensure dtype consistency
        target_cm_tensor = target_cm.as_tensor()
        
        # Compute coverage rate
        scaling = 1e10
        rate = tf.reduce_mean(tf.math.log(1. + target_cm_tensor * scaling))
        coverage_loss = -rate  # Note the negative sign to make it a loss
        # Combine SINR loss and coverage loss
        loss = weight_coverage * coverage_loss
    # Compute gradients and apply through the optimizer
    grads = tape.gradient(loss, tape.watched_variables())
    optimizer.apply_gradients(zip(grads, tape.watched_variables()))
    return loss

for step in range(num_steps):
    loss = train_step()
    print(f"Training step {step} - Loss: {loss.numpy():.2E}", end='\r')


In [ ]:
cm_new = scene.coverage_map(cm_center=xy_center + [z_centers[1]],
                            cm_orientation=[0,0,0],
                            cm_size=[100,50],
                            cm_cell_size=[0.1,0.1])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure and a set of subplots
fig, axs = plt.subplots(1,7, figsize=(50, 10))

cm_0 = 24 + 10*np.log10(np.where(cm_new[0]==0, 1e-12, cm_new[0]))
cm_1 = 24 + 10*np.log10(np.where(cm_new[1]==0, 1e-12, cm_new[1]))
cm_2 = 24 + 10*np.log10(np.where(cm_new[2]==0, 1e-12, cm_new[2]))
cm_3 = 24 + 10*np.log10(np.where(cm_new[3]==0, 1e-12, cm_new[3]))
cm_4 = 15 + 10*np.log10(np.where(cm_new[4]==0, 1e-12, cm_new[4]))
cm_5 = 24 + 10*np.log10(np.where(cm_new[5]==0, 1e-12, cm_new[5]))
cm_6 = 24 + 10*np.log10(np.where(cm_new[6]==0, 1e-12, cm_new[6]))

sns.heatmap(cm_0, ax=axs[0], vmin=-30, vmax=0)
sns.heatmap(cm_1, ax=axs[1], vmin=-30, vmax=0)
sns.heatmap(cm_2, ax=axs[2], vmin=-30, vmax=0)
sns.heatmap(cm_3, ax=axs[3], vmin=-30, vmax=0)
sns.heatmap(cm_4, ax=axs[4], vmin=-30, vmax=0)
sns.heatmap(cm_5, ax=axs[5], vmin=-30, vmax=0)
sns.heatmap(cm_6, ax=axs[6], vmin=-30, vmax=0)

# Adjust layout
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
scene.preview(coverage_map=cm_new, cm_tx=1, clip_at=3)

In [ ]:
import numpy as np

def calculate_sinr_matrix(signal_matrices, noise_power):
    # Convert the noise power to linear scale
    noise_linear = 10 ** (noise_power / 10)

    # Number of matrices (4 antennas) and dimensions of the matrix
    num_antennas = len(signal_matrices)
    rows, cols = signal_matrices[0].shape

    # Initialize the SINR matrix with zeros
    sinr_matrix = np.zeros((rows, cols))

    # Optionally store the index of the reference signal (strongest signal) for each point
    reference_signal_indices = np.zeros((rows, cols), dtype=int)

    # Loop through each point in the matrices
    for i in range(rows):
        for j in range(cols):
            # Extract the signal power values at this point from all antennas
            signal_values = [signal_matrices[k][i, j] for k in range(num_antennas)]
            if i == j and i == 200: print(signal_values)

            # Convert signal values from dB to linear scale
            signal_values_linear = [10 ** (sv / 10) for sv in signal_values]

            # Find the reference signal (the strongest signal)
            reference_signal_linear = max(signal_values_linear)
            reference_signal_index = signal_values_linear.index(reference_signal_linear)

            # Store the index of the reference signal (optional)
            reference_signal_indices[i, j] = reference_signal_index

            # Calculate interference (sum of all signals except the reference signal)
            interference_linear = sum(signal_values_linear) - reference_signal_linear

            # Calculate SINR at this point
            sinr_linear = reference_signal_linear / (interference_linear+noise_linear)

            if i == j and i == 200: print(reference_signal_linear, interference_linear, noise_linear, sinr_linear)

            # Convert SINR back to dB scale and store in the SINR matrix
            sinr_matrix[i, j] = 10 * np.log10(sinr_linear)

    return sinr_matrix, reference_signal_indices

sinr_matrix, _= calculate_sinr_matrix([cm_0,cm_1,cm_2,cm_3,cm_4,cm_5,cm_6], -999999999999999999)


In [ ]:
import numpy as np

# Define the threshold
T = 13

# Create binary matrix B
B = (sinr_matrix > T).astype(np.float32)

sns.heatmap(B)
plt.show()

In [ ]:
counter = 0
summ = []
for i,row in enumerate(sinr_matrix):
    if i < 300 and i > 180:
        for j,item in enumerate(row):
            if j < 580 and j > 140:
                if item > 0.5:
                    summ.append(item)
                if item > 0.5 and item < 5: print(item,i,j)
            
            

print(min(summ),max(summ),np.mean(summ))

In [ ]:
# Print the resulting SINR matrix
print("SINR Matrix (in dB):")
sns.heatmap(sinr_matrix)
plt.show()


In [ ]:
from sionna.rt.utils import r_hat, rotation_matrix, theta_phi_from_unit_vec, expand_to_rank
import tensorflow as tf
import numpy as np

In [ ]:
# Convert the list to a NumPy array
angles = np.array([-2.80684, 0.36026, 0.027153])

# Now, pass the NumPy array to the rotation_matrix function
rot_mat = tf.transpose(rotation_matrix(angles))

In [ ]:
np.arctan2(rot_mat[1][1],rot_mat[0][1]) * 180 / 3.14

In [ ]:
np.arcsin(-rot_mat[2][1]) * 180 / 3.14